# Environment simulations using ODEs

Environment dynamics with agents with some understanding of how the environment
works can be simulated as a system of ordinary differential equations. Below is
an example of how to run simulations like that using `gxr.envir` package.

For now (this will be extended later) our prototype analysis focuses on the effects
of:

* Time horizon in which agents try to predict consequences of their actions,
  i.e. the effects of how much resource they extract from the environment.
  This is controlled by `horizon` parameter, which defines (approximately) the
  length of the time horizon of foresight expressed in terms of the characteristic
  timescale of the environment (the amount of time it needs to regenerate from 5%
  to 95% of its carrying capacity).
  * `horizon` $\in (0, \infty)$
  
* For now the default number of agents is ``4`` as this is the planned number
  of participants in VR experiments.

There are of course several other control parameters, but for now they are all set
to reasonable defaults, and therefore we are not interested in them.

## Running a single simulation

Below is an example of how to run a single simulation and visualize its results.

In [ ]:
from gxr.envir import Config, DynamicsPlotter, EnvirDynamics, EnvirModel

# Configuration for the game; only value to override must be provided;
# all other options are set to reasonable defaults.
# Use '.from_disk()' method to populate a config with value from a '.toml' file.
params = {
    "foresight": {
        "horizon": .01,
    },
}
config   = Config(params)
model    = EnvirModel(**config["model"])
dynamics = EnvirDynamics(model)

We use `dynamics` object to run a simulation of environment-agents dynamics.
The first argument is the length of the run expressed in terms of the characteristic
timescale.

The solution object, `sol`, exposes a `.get_arrays()` method returning a names tuple with 4 arrays:

1. ``T`` - Time grid.
2. ``E`` - Environment states in time.
3. ``P`` - Agents profits in time, that is, the cumulative resources extracted 
           from the minus the sustenance costs.
4. ``H`` - Individual harvesting rates of agents in time. These are the intensities
           of resource extraction (and the actual gain is proportional to the
           harvesting intensity and the current state of the environment).

In [ ]:
sol  = dynamics.run(30, progress=True)
T, E, P, H = sol.get_arrays()

# Solver message
sol.ode.message

**NOTE.** Sometimes solver may not reach the end of the integration interval.
This happens almost exclusively when the environment state becomes low enough,
to be dominated by floating point errors.

It is probably best to rerun simulations for which ``sol.ode.success is not True``.

It is also quite easy to visualize the results of a single simulation run using
out `DynamicsPlotter` class.

In [ ]:
plotter = DynamicsPlotter(dynamics, sol)
fig, axes = plotter.subplots(nrows=3, figsize=(8, 8))

plotter.plot_state(axes[0], show_vicious=True, show_perceived=True)
plotter.plot_harvesting(axes[1])
plotter.plot_utilities(axes[2])
fig.tight_layout()
fig

**NOTE.** Currently, an identity utility function is used, so utilities are
equal to profits (i.e. extracted resource minus sustenance cost).

## Running multiple simulations for generating training data

Here we discuss an example of how the problem of using simulations to generate
training data can be approached. It is of course, is not the only possible solution,
but rather a guide for how other custom approaches can be designed.

For now, we focus on effects of the `horizon` parameter, so we will study over
some grid from `.01` to `20`. For each value we generate `n_reps` instances of
dynamics and save the results using the mean final profit of agents as the target
variable that, in principle, should be maximized by a learned policy.

Currently, there is no easy way of modifying `horizon` (or any other control
parameter for that matter) during a simulation, so it is not clear generate data
from which effects of interventions (i.e. changes of `horizon`) could be inferred.
However, this problem can be bypassed using a trick.

For each instance of dynamics we will sample one (or more) sub-trajectories,
i.e. from time $t_1$ to $t_2$, with lengths selected uniformly at random.
And since the dynamics have a form of a continuous time Markov chain, each such
sample will describe an effect of a having a particular value of `horizon`
conditional on a particular set of initial conditions 
(environment state and current agents' profits and individual harvesting rates)
over a particular time interval.

And it is showed below, it is quite easy to generate data like that.

In [ ]:
import random

import numpy as np
from tqdm import tqdm

## FIRST DEFINE SOME BASIC PARAMETERS OF THE SIMULATION
## The values provided here are of course only examples.

HORIZON   = np.linspace(.01, 10, 20)
T_MAX     = 50  # simulation run time in terms of the characteristic timescale
N_REPS    = 10   # number of simulation runs per a parameter configuration
N_SAMPLES = 10   # number of samples from a single simulation run

## Set random seed for sampling reproducibility
random.seed(1010103)

results = []

for horizon in tqdm(HORIZON):
    for _ in tqdm(range(N_REPS), leave=False):
        params = {"foresight": {"horizon": horizon}}
        config   = Config(params)
        model    = EnvirModel(**config["model"])
        dynamics = EnvirDynamics(model)

        sol = dynamics.run(T_MAX, progress=False)
        result = {
            "horizon": horizon,
            "data": sol.get_arrays()
        }

        for _ in range(N_SAMPLES):
            ## Here we get a sample and make time and profit values relative,
            ## i.e. starting time and profits are 0.
            sample = sol.get_arrays(sample=True).relative
            result.setdefault("samples", []).append(sample)

        results.append(result)

This data representation can be easily turned, for instance, into `pandas` data frames
allowing for easy computation of target values, aggregation and visualization.

Below is a graph of target values with respect to horizon lengths.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

sim = (
    pd.DataFrame(results)
    .assign(target=lambda df: df["data"].map(lambda A: A.P[..., -1].mean()))
)

data = sim
mean_target = data.groupby("horizon")["target"].mean()

fig, ax = plt.subplots()
ax.plot(mean_target.index, mean_target, color="red", label="mean target value")
ax.scatter(data["horizon"], data["target"])
ax.set_xlabel("Time horizon")
ax.set_ylabel("Target (mean final profit)")
ax.legend()
fig

The correlation between `horizon` and final average profits is much weaker, but this
is due to the fact that samples are of different lengths and have different starting
conditions.

Crucially, an individual sample provides information on a target value after a given
time starting from initial conditions, that is, environment state and agents' profits
and harvesting rates.

In [ ]:
samples = pd.DataFrame({
    "horizon": sim["horizon"],
    "time": sim["samples"].map(lambda S: [s.T.max() for s in S]),
    "target": sim["samples"].map(lambda S: [s.P[..., -1].mean() for s in S])
}).explode(["time", "target"])

data = samples
# data = data[data["time"] > 5000]
mean_target = data.groupby("horizon")["target"].mean()

fig, ax = plt.subplots()
ax.plot(mean_target.index, mean_target, color="red", label="mean target value")
ax.scatter(data["horizon"], data["target"])
ax.set_xlabel("Time horizon")
ax.set_ylabel("Target (mean final profit)")
ax.legend()
fig

Below is a more detailed exposition of information a single sample contains.

In [ ]:
# ruff: noqa
sample = results[0]["samples"][0]

sample.T      # time grid of the sample starting from 0
sample.T[-1]  # time interval length

sample.E      # environment states in time

sample.P                  # agents' profits in time
sample.P[..., -1].mean()  # average final agents' profit aka target

sample.H  # agents' individual harvesting rates in time

In principle, the information listed above may be used for learning the relation between
`horizon`, initial conditions and expected target values.

### Some caveats

* In this approach the learning problem actually reduces to a supervised scenario.
* Samples taken from the same dynamics will be somehow correlated. It is hard to know
  for sure, but probably this should not be too much of a problem for learning as long
  as the number of independent simulated dynamics is high enough.